tensorflow api 사용 -> version 차이 맞춰야 하고 중간 과정 이해 필요함... (해볼예정)

In [1]:
import pandas as pd 

new_df = pd.read_csv(os.path.join("..","..","data","TA_User_Reviws_Korea_all_v2_new_df.csv"))
new_df = new_df.drop(['location_name'], axis=1)
new_df.head()

,Unnamed: 0,userID,users_mean_rating,user_reviewcount,locationId,rating,placeType,createdDate,islocal,photonum
0,0,0,5.0,1,3656150,5,ACCOMMODATION,20140121,0,0
1,1,1,5.0,1,301253,5,ACCOMMODATION,20161005,0,2
2,2,2,5.0,1,306139,5,ACCOMMODATION,20140324,0,0
3,3,3,4.2,10,4071058,5,EATERY,20161113,1,0
4,4,3,4.2,10,11959139,5,EATERY,20170405,1,1


1. wide 부분에 대한 features 선택 : 사용하길 원하는 sparse base column 과 crossed column 들을 선택한다  
2. deep 부분에 대한 features 선택 : 연속된 열, 각 분류 열의 embedding dimension, 그리고 hidden layer 크기를 선택한다  
3. 이들을 Wide & Deep 모델에 적용한다(DNNLinearCombinedClassifier)

# Base Feature Columns 정의  
사용할 base categorical feature column 과 continuous feature column 들을 정의하자. 이들 base column 들은 모델의 wide 부분과 deep 부분에 모두 사용될 building block 들이 된다.

In [8]:
import tensorflow as tf

# Categorical base columns.
placeType = tf.keras.layers.sparse_column_with_keys(column_name="placeType", keys=["ACCOMMODATION", "EATERY"])

# Continuous base columns.
userID = tf.contrib.layers.real_valued_column("userID")
users_mean_rating = tf.contrib.layers.real_valued_column("users_mean_rating")
user_reviewcount = tf.contrib.layers.real_valued_column("user_reviewcount")
locationId = tf.contrib.layers.real_valued_column("locationId")
createdDate = tf.contrib.layers.real_valued_column("createdDate")
photonum = tf.contrib.layers.real_valued_column("photonum")

AttributeError: module 'tensorflow_core.keras.layers' has no attribute 'sparse_column_with_keys'

# The Wide Model: Linear Model with Crossed Feature Columns  
wide 모델은 sparse 하고 crosseed feature column 들의 다양한 집합의 선형 모델이다

In [ ]:
wide_columns = [
  placeType, 
  tf.contrib.layers.crossed_column([education, occupation], hash_bucket_size=int(1e4)),
  tf.contrib.layers.crossed_column([native_country, occupation], hash_bucket_size=int(1e4)),
  tf.contrib.layers.crossed_column([age_buckets, race, occupation], hash_bucket_size=int(1e6))]

crossed feature column 들의 Wide 모델은 feature들 간의 sparse interactions(상호작용들)을 효율적으로 저장할 수 있다. 그럼에도, crossed feature columns 의 한가지 제한되는 점은 학습된 데이터에 나타나지 않는 feature 조합들을 생성해내지 못한다는 것이다. 이를 수정하기 위해 embeddings 를 포함한 deep 모델을 추가

# The Deep Model: Neural Network with Embeddings
이전 그림에서 보이는 것과 같이 deep 모델은 feed-forward 신경망이다. 각각의 sparse, high-dimensional(고차원의) categorical feature 들은 종종 embedding 벡터라 불리는 low-dimensional(저차원의), dense real-valued 벡터로 변환된다. 이 low-dimensional(저차원의) dense embedding 벡터들은 continuouse feature 들로 연계되어 변환되고, 계속해서 다음 신경망의 hidden layers 로 연결된다. embedding value 들은 무작위로 초기화되고, 학습 loss(손실)을 최소화하는 다른 모든 모델 파라미터들과 함께 학습된다.    
  
embedding_column 을 이용하여 categorical column 들을 위한 embeddings 를 설정할 것이다. 그리고 이들을 continuous columns 과 연관지을 것이다.



In [ ]:
deep_columns = [
  tf.contrib.layers.embedding_column(placeType, dimension=8),
  userID, users_mean_rating, user_reviewcount, locationId, createdDate, photonum]


dense embeddings 를 통해, deep 모델은 학습된 데이터에서 이전에 보여지지 못한 feature 쌍들에 대한 생성을 더 잘 할 수 있고 예측할 수 있다. 그러나 두 feature column 간의 기본 interaction matrix(상호작용 메트릭스)가 sparse 하고 high-rank 일 때, feature column 들에 대한 low-dimensional representation 들의 효율적인 학습은 어렵다. 이러한 경우, 대부분의 feature 쌍 들간 interaction(상호작용)은 몇몇을 제외하는 0(zero) 이 되어야 한다. 하지만 dense embeddings 는 모든 feature 쌍들에 대해 none-zero(0 이 아닌) 예측을 이끌어 냄에 따라, 지나치게 일반화할 수 있다. 반면, crossed features 선형모델은 이러한 "exception rules"(예외들)을 더 적은 모델 파라미터들을 이용해 효율적으로 저장할 수 있다.
이제 어떻게 wide 모델과 deep모델을 함께 학습시키고 그들 각각의 강점과 약점을 보완해 가는지 살펴보자.

# Wide and Deep 모델을 하나로 조합하기
wide 모델과 deep 모델들은 예측으로써 그들의 마지막 출력 로그 나머지(final output log odd) 의 합에 의해 조합된다. 그 다음 로지스틱 loss 함수의 예측에 전달된다. 모든 그래프 정의와 변수 할당량들은 이미 내부에서 다뤄졌기 때문에, DNNLinearCombinedClassifier 를 만들어 볼 필요가 있다.:

In [ ]:
import tempfile
model_dir = tempfile.mkdtemp()
m = tf.contrib.learn.DNNLinearCombinedClassifier(
    model_dir=model_dir,
    linear_feature_columns=wide_columns,
    dnn_feature_columns=deep_columns,
    dnn_hidden_units=[100, 50])

# 모델 학습하기와 평가하기

In [ ]:
import pandas as pd
import urllib

# Define the column names for the data sets.
COLUMNS = ["placeType" ,"userID", "users_mean_rating", "user_reviewcount", "locationId",
                      "createdDate", "photonum"]
LABEL_COLUMN = 'rating'
CATEGORICAL_COLUMNS = ["placeType"]
CONTINUOUS_COLUMNS = ["userID", "users_mean_rating", "user_reviewcount", "locationId",
                      "createdDate", "photonum"]


def input_fn(df):
  # Creates a dictionary mapping from each continuous feature column name (k) to
  # the values of that column stored in a constant Tensor.
  continuous_cols = {k: tf.constant(df[k].values)
                     for k in CONTINUOUS_COLUMNS}
  # Creates a dictionary mapping from each categorical feature column name (k)
  # to the values of that column stored in a tf.SparseTensor.
  categorical_cols = {k: tf.SparseTensor(
      indices=[[i, 0] for i in range(df[k].size)],
      values=df[k].values,
      shape=[df[k].size, 1])
                      for k in CATEGORICAL_COLUMNS}
  # Merges the two dictionaries into one.
  feature_cols = dict(continuous_cols.items() + categorical_cols.items())
  # Converts the label column into a constant Tensor.
  label = tf.constant(df[LABEL_COLUMN].values)
  # Returns the feature columns and the label.
  return feature_cols, label

def train_input_fn():
  return input_fn(df_train)

def eval_input_fn():
  return input_fn(df_test)

데이터를 읽고 난 후, 모델을 학습하고 평가할 수 있다:

In [ ]:
m.fit(input_fn=train_input_fn, steps=200)
results = m.evaluate(input_fn=eval_input_fn, steps=1)
for key in sorted(results):
  print "%s: %s" % (key, results[key])